# Monte Carlo Methods

- Here we do not assume complete knowledge of the environment
- Monte Carlo methods require only experience -sample sequences of states, acitons and rewards from actual or simulated interaction with an environment.  Learning from actual experience is striking because it requires no prior knowledge of the environment's dynamics, yet can still attain oprimal behavior.
- Learning from simulated experience is also powerful although it requires a model
    - The model only nees to generate sample transitions, not the complete probability distributions of all possible transitions that is required for DP.  
    - In surprisingly many cases it is easy to generate experience sampled according to the desired probability distribution but infeasible to obtain the distributions in explicit form.
- Monte Carlo methods are ways of solving the RL problem based on averaging sample returns.  
- To ensure that well-defined returns are available, here we define MC methods only for episodic tasks.
- Only on the completion of an episode are value estimates and policies changed.
    - MC methods can thus be incrmemental in an episode-by-episode sense, but not in a step-by-step (online) sense
- Whereas in DP we computed value functions from knowledge of the MDP, here we learn value functions from sample returns with the MDP.  The value functions and corresponding policies still interact to attain optimality in essentially the same way (GPI).

# 5.1: Monte Carlo Prediction

- We begin by considering MC methods for learning the state-value function for a given policy
- Recall that the value of a state is the expected return- expected cumulative future discounted reward- starting from that state.  An obvious way to esitmate it from experience, then, is simply to average the returns observed after visits to that state.  As more returns are observed, the averge should converge to the expected value.  This idea undlies all MC methods
- Suppose we wish to estimate $v_{\pi}(s)$ given a set of episodes obtained by following $\pi$ and passing through s.
- Each occurrence of stae s in an episode is called a visit to s
    - The first time a state is visited in an episode is referred to as the first visit.
- The first-visit MC method estimates $v_{\pi}(s)$ as the average of the returns following first visits to s, whereas the every-visit MC method averages the returns following all visits to s
    - We will focus on first-visit MC in this chapter
- Both first-visit MC and every-visit MC converge to $v_{\pi}(s)$ as the number of visits (or first visits) to s -> inf
- In this case, each return is an independent, identically distributed estimate of $v_{\pi}(s)$ with finite variance
- The ability of MC methods to work with sample episodes alone can be a significant advantage even when one has complete knowledge of the environment's dynamics
- The backup diagram for MC estimation of $v_{\pi}(s)$ has the state node at the root and below all the sampled transitions until the end of the episode.
- An important face about MC methods is that the estimates for each state are independent.  The estimate for one state does not build upon the estimate of any other state, as in the case in DP.  In other words, MC methods do not bootstrap
- Note that the computational expense of estimating the value of a single state is independent of the number of states.
- If one is interested in only the value at one point, or any fixed small set of points, then this MC method can be far more efficient than the iteraive method based on local consistency.
- Three advantages of MC methods over DP methods:
    - ability to learn from actual experience
    - ability to learn from simulated experience
    - estimating value of state is independent of total number of states, which is attractive when we only care about a subset of states 

In [ ]:
# PSEUDOCODE: First visit MC prediction, for estimating V ~= V_PI
'''
Input: a policy PI to be evaluated

Initialize:
    V(s) in R, arbitrarily, for all s in S
    Returns(s) <- an empty list, for all s in S

Loop forever (for each episode):
    Generate an episode following PI: S_0, A_0, R_1, S_1, A_1, R_2, ..., S_(T-1), A_(T-1), R_T
    G <- 0
    Loop for each step of episode, t = T-1, T-2, ..., 0:
        G <- gamma*G + R_(t+1)
        Unless S_t appears in S_0, S_1, ..., S_(t-1):
            Append G to Returns(S_t)
            V(S_t) <- average(Returns(S_t))
'''


# 5.2: MC Estimation of Action Values

- If a model is not available, then it is particularly useful to estimate action values rather than state values.  With a model, state values alone are sufficient to determine the policy; one simply looks ahead one step and chooses whichever action leads to the best combination of reward and next state. Without a model, however, state values alone are not sufficient.  One must explicityly estimate the value fxn of each action in order for the values to be useful in suggesting the policy.
- The policy evalueation problem to estimate action values is to estimate $q_{\pi}(s, a)$, the expected return when starting in state s, taking action a, and thereafter following policy $\pi$
- The MC methods to estimate action values is similar to the method to estimate state values
- A state-action pair s,a is said to have been visited in an episode if ever the state s is visited and action a is taken in it
- Every-visit MC method estimates the value of a state-action pair as the average of the returns that have followed all the visits to it.
- First-visit MC mehtod averages returns following the first time in each episode that the state was visited and the action was selected.
- A complication is that many state-action pairs may never be visited
- One way to deal with this si to specifyin that episodes start in a state-action pair, and that every pair has a nonzeros probability of being selected as the start.  This guarantees that all state-acition pairs will be visited an infinite number of times as the number of episode goes to inf. We call this the assumption of exploring starts.
    - This assumption can sometimes be useful but cannot be relied on in general, particularly when learning from actual interacion with an environment.
        - In this case, the starting conditions are unlikely to be helpful
- The most common alternative approach to assuring that all state-action pairs are encountered is to consider only policies that are stochastic with nonzero probability of selecting all acitons in each state

# 5.3: MC Control

- The overall idea for using MC methods to approximate optimal policies is to proceed according to the same idea of GPI
- If you assume exploring starts and that we observe an infinite number of episodes then we can follow the same Policy Improvement method from DP to generate an optimal policy given only sample episodes and no other knowledge of the environment's dynamics
- We can avoid assumption of initite number of episodes by giving up on trying to complete policy evaluation befor returning to policy improvement.  On each evaluation step we move the value function toward $q_{\pi_{k}}$ but we do not expect to actually get close except over many steps.
- For MC policy iteration it is natural to alternate between evaluation and improvement on an episode by episode basis.
    - After each episode, the observed returns are used for policy evaluation, and then the policy is improved at all states visited in the episode.


# 5.4: MC Control without Exploring Starts

- The only general way to ensure that all actions are selected infinitely often is for the agent to continue to select them.  There are two approaches to ensure this:
    - On-policy methods: these attempt to evaluate or improve the policy that is used to make decisions
    - Off-policy methods: these evaluate or improve a policy different from that used to generate the data
- In on-policy control methods, the policiy is generally soft , meaning that $\pi(a|s) > 0$ for all s in S and all a in A(s), but gradually shifted closer and closer to a deterministic optimal policy.
- GPI dows not require that the policy be taken all the way to a greedy polkcy, only that is be moved toward a greedy policy

# 5.5: Off-policy Prediction via Importance Sampling

- All learning control methods face a dilemma: they seek to learn action values conditional on subsequent optimal behavior, but they need to behave non-optimally in order to explore all actions(to find the optimal actions).  How can they learn about the optimal policy while behaving according to an exploratory policy?
    - The on-policy approach learns action values not for the optimal policy, but for a near-optimal policy that still explores
    - Another approach is to use off-policy learning
- Off-policy learning: use two policies, one that is learned about and that becomes the optimal policy, and one that is exploratory and is used to generate behavior.
    - The policy being learned about is called the target policy, and the policy being used to generate behavior is called the behavior policy
- On-policy methods are generally simpler and considered first
- Off-policy methods require additional concepts and notation, and because the data is due to a different policy, off-policy methsds are often of greater variance and are slower to converge.
- Notation: we refer to the target policy as $\pi$ and the behavior policy as b
- We will start with the prediction problem: we are trying to estimate $v_{\pi}$ or $q_{\pi}$, but all we have are episodes following another policy b
    - In order to use episodes from b to estimate values for $\pi$ we require that every action taken under $\pi$ is also taken under b.  This is called the assumption of coverage
        - $\pi(a|s) > 0$ implies $b(a|s) > 0 $
- In control, the target policy is typically the deterministic greedy policy with respect to the current estimate of the action-value fxn.  This policy becomes a deterministic optimal policy while the behavior policy remains stochastic and more exploratory, for example, an epsilon-greedy policy.
- Almost all off-policy methods utilize importance sampling, a general technique for estimating expected values under one distribution given samples from another.
- We apply impotance sampling to off-policy learning by weighting returns acccording to the relative probability of their trajectories occuring under the target and behavior policies, called the importance-sampling ratio, $\rho_{ t:T-1} = \Pi_{k=t \rightarrow T-1} \frac{\pi(A_{k}|S_{k})}{b(A_{k}|S_{k})}$